In [5]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from qiskit.circuit.library import efficient_su2
from Quantum_Games import game_circuit, player, prisoners_dilemma_payoff_calculator, get_aer_result

In [3]:
params = [(i*4*np.pi, i*4*np.pi) for i in np.linspace(0, 1, 10)]
strategies = [efficient_su2(1, reps=0).assign_parameters(parameters=param) for param in params]
n_players = 2

In [53]:
from qiskit.circuit import Gate

long = Gate([[1, 0], [0, 1]])
short = Gate([[0, 1], [1, 0]])
quantum_long = Gate([[1j, 0], [0, -1j]])
quantum_short = Gate([[0, -1j],[-1j, 0]])
strategies = [long, quantum_long, quantum_short, short]


TypeError: Gate.__init__() missing 2 required positional arguments: 'num_qubits' and 'params'

In [54]:
def payoff(theta):
    players = []

    probs = theta.tolist()
    norm = sum(probs)

    if norm >= 1:
        return norm
    
    probs.append(1-norm)
    print(probs)
    for i in range(n_players):
        players.append(player(name = f'player_{i}', strategies=strategies, probabilities=probs))

    #num_strats = len(strategies)
    #probs_assignments = [theta[i:i + num_strats] for i in range(0, len(theta), num_strats)]
    #for i in range(len(probs_assignments)):
        #players.append(player(name = f'player_{i}', strategies=strategies, probabilities=probs_assignments[i]))
    
    circuit = game_circuit(players=players)
    result = get_aer_result(circuit)
    result_df = pd.DataFrame(result)
    payoffs = prisoners_dilemma_payoff_calculator(result_df)
    ave_payoff = -payoffs.values.mean()
    print(ave_payoff)
    return ave_payoff


In [55]:
x0 = np.random.rand(2*n_players)
x0 = x0/sum(x0)
x0 = x0 [:-1]
print(x0)
constraints = {'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)}
bounds = [(1e-6, 1.0) for _ in range(len(x0))] 
result = minimize(fun=payoff,
                  method='SLSQP',
                  bounds=bounds,
                  constraints=constraints,
                  x0=x0)

[0.32749205 0.35186815 0.26847821]
[0.32749205015000105, 0.3518681469759046, 0.2684782104266488, 0.05216159244744545]
-0.4885
[0.32749206505116224, 0.3518681469759046, 0.2684782104266488, 0.052161577546284255]
-0.4945
[0.32749205015000105, 0.3518681618770658, 0.2684782104266488, 0.052161577546284255]
-0.502
[0.32749205015000105, 0.3518681469759046, 0.26847822532781, 0.052161577546284255]
-0.542


In [56]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.4885
       x: [ 3.275e-01  3.519e-01  2.685e-01]
     nit: 5
     jac: [-4.027e+05 -9.060e+05 -3.590e+06]
    nfev: 4
    njev: 1

In [57]:
def generate_initial_simplex(theta_0, perturbation_size=0.05):
    """
    Generates an initial simplex for the Nelder-Mead algorithm.

    Parameters:
        theta_0 (np.array): Initial guess (starting point) as a 1D array.
        perturbation_size (float): Size of the perturbation for each dimension.

    Returns:
        np.array: A (n+1, n) array representing the initial simplex.
    """
    n = len(theta_0)  # Number of dimensions
    simplex = np.zeros((n + 1, n))  # Initialize simplex matrix

    # First vertex is the initial guess
    simplex[0] = theta_0

    # Generate the remaining vertices by perturbing each dimension
    for i in range(n):
        vertex = theta_0.copy()
        vertex[i] += perturbation_size  # Perturb the i-th dimension
        simplex[i + 1] = vertex

    return simplex

In [62]:
x0 = np.random.rand(2*n_players)
x0 = x0/sum(x0)
x0 = x0 [:-1]
initial_simplex = generate_initial_simplex(x0, perturbation_size=0.5)
result = minimize(fun=payoff,
                  x0 = x0,
                  method='Nelder-Mead',
                  bounds = bounds,
                  tol=1e-2,
                  options = {'maxiter': 100,
                             'initial_simplex' : initial_simplex,
                            }
                 )

[0.03537867355112473, 0.34323514354277673, 0.43154335682278616, 0.18984282608331238]
-0.055
[0.5909342291066804, 0.06545736576499894, 0.14384845227426207, 0.19975995285405856]
-0.0605
[0.8687120068844582, 1e-06, 1.0000000000287557e-06, 0.1312859931155418]
-0.0
[0.12797126614371745, 0.3802721805798136, 1e-06, 0.4917555532764689]
-0.378
[1e-06, 0.3987906990983321, 1e-06, 0.601207300901668]
-0.372
[0.1341441056498902, 1e-06, 0.38359420606469885, 0.4822606882854109]
-0.3875
[0.016860155032606183, 1e-06, 0.5753908090970483, 0.40774803587034547]
-0.374
[0.5333210603824006, 1e-06, 1e-06, 0.4666769396175994]
-0.5515
[0.7822922537980386, 1e-06, 1e-06, 0.21770574620196137]
-0.0
[1e-06, 0.18805875462154348, 0.11188235176887046, 0.700057893609586]
-0.5
[0.31700617787780977, 1e-06, 0.3303173718890462, 0.35267545023314406]
-0.34
[0.17522999407724055, 0.22147954939349737, 0.08258009297226154, 0.5207103635570005]
-0.504
[0.3382239306565373, 0.27302520267669395, 1e-06, 0.3887498666667687]
-0.4845
[0.28

In [63]:
result

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: -0.5765
             x: [ 2.451e-01  2.419e-01  2.109e-02]
           nit: 100
          nfev: 256
 final_simplex: (array([[ 2.451e-01,  2.419e-01,  2.109e-02],
                       [ 2.451e-01,  2.419e-01,  2.109e-02],
                       [ 2.451e-01,  2.419e-01,  2.109e-02],
                       [ 2.451e-01,  2.419e-01,  2.109e-02]]), array([-5.765e-01, -5.655e-01, -5.460e-01, -5.065e-01]))